In [43]:
# !git clone https://github.com/MrRezaeiUofT/Amazon-Product-Recommender.git
# !cd Amazon-Product-Recommender && zip -s- Dataset.zip -O /content/AmazonDataset
# !unzip AmazonDataset.zip
# !mkdir AmazonMusicRatings && cd AmazonMusicRatings && \
# cp /content/train.json/train.json . && \
# cp /content/test.json/test.json . && \
# cp /content/rating_pairs.csv .
# %cd AmazonMusicRatings

/content/AmazonMusicRatings


In [96]:
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import tensorflow.keras.backend as K
import tensorflow as tf
import string
import snowballstemmer
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
from sklearn.manifold import TSNE
import numpy as np
from collections import defaultdict
import json
from collections import defaultdict
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Dropout,Flatten,Dense,TimeDistributed,Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
np.random.seed(0)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from collections import defaultdict
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils import resample
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Dropout,Flatten,Dense,TimeDistributed,Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
np.random.seed(0)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from collections import defaultdict
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Dropout,Flatten,Dense,TimeDistributed,Input, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, RMSprop
np.random.seed(0)
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
MAX_WORD_TO_USE = 1000 # how many words to use in training
MAX_WORD_TO_USE_other=100
MAX_LEN = 100 # number of time-steps.

In [ ]:
def Text_cleaner(Data):
    HoleText = []
    separator = ' '
    for i in range(Data.shape[0]):
        HoleText.append(separator.join(text_to_word_sequence(str(Data[i]))))
    x_c = HoleText
    return x_c

In [ ]:
def Text_length(Data):
    HoleText = []
    for i in range(Data.shape[0]):
        HoleText.append(text_to_word_sequence(str(Data[i])))
    x_c = HoleText
    x_c_l=[len(x_c[i]) for i in range(len(x_c))]
    return x_c_l

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy

    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss

    return loss

In [ ]:
def weighted_mse(weights):
    """
    A weighted version of MSE
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes

    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.

        model.compile(loss=loss,optimizer='adam')
    """

    weights = K.variable(weights)

    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        # y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = tf.reduce_logsumexp(tf.pow(y_true - y_pred,2) * weights)

        return loss

    return loss

In [ ]:
def clean_text(text):
    ## Remove puncuation
    text = text.translate(string.punctuation)

    ## Convert words to lower case and split them
    text = text.lower().split()

    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    # Stemming
    text = text.split()
    stemmer = nltk.stem.SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [48]:
# allRatings = []
# userRatings = defaultdict(list)

# with open('train.json', 'r') as train_file:
#     for row in train_file:
#         data = json.loads(row)
#         r = float(data['overall'])
#         allRatings.append(r)
#         userRatings[data['reviewerID']].append(r)

# globalAverage = sum(allRatings)/len(allRatings)
# userAverage = {}
# for u in userRatings:
#     userAverage[u] = sum(userRatings[u]) / len(userRatings[u])

# predictions = open('rating_predictions.csv', 'w')
# for l in open('rating_pairs.csv'):
#     if l.startswith('userID'):
#         #header
#         predictions.write(l)
#         continue
#     u,p = l.strip().split('-')
#     if u in userAverage:
#         predictions.write(u + '-' + p + ',' + str(userAverage[u]) + '\n')
#     else:
#         predictions.write(u + '-' + p + ',' + str(globalAverage) + '\n')
# predictions.close()

In [88]:
data = []
bad_records = 0
with open('train.json') as f:
    for line in f:
      try:
        data.append(json.loads(line))
      except:
        bad_records+=1
        pass
print('Bad records skipped: {}'.format(bad_records))

df = pd.DataFrame.from_records(data).reset_index()
df.head()

Bad records skipped: 53


,index,overall,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,0,4.0,"08 24, 2010",u04428712,"So is Katy Perry's new album ""Teenage Dream"" c...",Amazing that I Actually Bought This...More Ama...,1282608000,Pop,$35.93,p70761125,85559980,NaN
1,1,5.0,"10 31, 2009",u06946603,"I got this CD almost 10 years ago, and given t...",Excellent album,1256947200,Alternative Rock,$11.28,p85427891,41699565,NaN
2,2,4.0,"10 13, 2015",u92735614,I REALLY enjoy this pairing of Anderson and Po...,"Love the Music, Hate the Light Show",1444694400,Pop,$89.86,p82172532,24751194,NaN
3,3,5.0,"06 28, 2017",u35112935,Finally got it . It was everything thought it ...,Great,1498608000,Pop,$11.89,p15255251,22820631,NaN
4,4,4.0,"10 12, 2015",u07141505,"Look at all star cast. Outstanding record, pl...",Love these guys.,1444608000,Jazz,$15.24,p82618188,53377470,NaN


In [89]:
rate_pair_df = pd.read_json('test.json', lines=True)
rate_pair_df.head()

,reviewTime,reviewerID,reviewText,summary,unixReviewTime,category,price,itemID,reviewHash,image
0,"03 26, 2015",u32476110,"Fantastic mix of ""old school"" with a creative ...","Fantastic mix of ""old school"" with a creative ...",1427328000,Pop,$14.98,p76243483,20167847,NaN
1,"05 15, 2017",u36732410,Update: Indications\nThere are various opinion...,Digitally Extracted Stereo (DES) Rules!,1494806400,Pop,$15.16,p92485419,30527605,NaN
2,"06 4, 2015",u85385007,This album provides a new twist on old Sammy H...,Excellent unplugged album,1433376000,Pop,$7.37,p40031588,12169432,NaN
3,"04 23, 2009",u30715529,(Symbol) can be considered as another masterpi...,another masterpiece,1240444800,Pop,$12.45,p88719785,55648615,NaN
4,"09 15, 2000",u95909892,Many would think this album is good only becau...,True Classic Rock,968976000,Alternative Rock,$2.07,p59188380,9520938,NaN


In [90]:
rate_pair_id = pd.read_csv('rating_pairs.csv')
rate_pair_id.head()

,userID-itemID,prediction
0,u32476110-p76243483,NaN
1,u36732410-p92485419,NaN
2,u85385007-p40031588,NaN
3,u30715529-p88719785,NaN
4,u95909892-p59188380,NaN


In [91]:
compensate_number = df.overall.value_counts()[5]
compensate_number

81710

In [92]:
# MaxWordLength=50
# Batch_Size = 20000
# Dp_rate=0.2
# Epochs=200
# vocabulary_size = 2000

In [121]:
def preprocess(train, test, vocabulary_size):

  train.reset_index(drop=True, inplace=True)
  test.reset_index(drop=True, inplace=True)

  # combine summary and review text columns
  text = []
  for i in range(train.shape[0]):
    text.append(clean_text(str(train['summary'][i]) + str(train['reviewText'][i])))
  for i in range(test.shape[0]):
    text.append(clean_text(str(test['summary'][i]) + str(test['reviewText'][i])))

  # text tokenization
  tokenizer = Tokenizer(num_words=vocabulary_size)
  tokenizer.fit_on_texts(text)
  F_text = tokenizer.texts_to_matrix(text, mode='freq')

  F_text = F_text[:train.shape[0],:]
  F_text_te = F_text[train.shape[0]:,:]

  # time normalization
  F_time = (train.unixReviewTime.values - train.unixReviewTime.min())/(train.unixReviewTime.max() - train.unixReviewTime.min())
  F_time = F_time.reshape([-1,1])

  # dummy encoding of category column
  F_cat = pd.get_dummies(train.category).values

  newD = pd.DataFrame(np.concatenate([train['itemID'].values, test['itemID'].values], axis=0), columns=['itemID'])
  newD['reviewerID'] = np.concatenate([train['reviewerID'].values, test['reviewerID'].values], axis=0)
  newD['itemID'] = newD['itemID'].groupby(newD['itemID']).transform('count')
  newD['reviewerID'] = newD['reviewerID'].groupby(newD['reviewerID']).transform('count')

  RID_temp = pd.get_dummies(newD['reviewerID']).values
  F_newD_reviewerID = RID_temp[:train.shape[0],:]
  F_newD_reviewerID_te = RID_temp[train.shape[0]:,:]

  IID_temp = pd.get_dummies(newD['itemID']).values
  F_newD_itemID = IID_temp[:train.shape[0],:]
  F_newD_itemID_te = IID_temp[train.shape[0]:,:]

  vocab_size_RID=RID_temp.max()+1
  vocab_size_IID=IID_temp.max()+1

  return F_text, F_time, F_cat, F_newD_reviewerID, F_newD_itemID, F_newD_itemID_te, vocab_size_RID, vocab_size_IID

In [112]:
temp_train = df.sample(1000).reset_index(drop=True)
temp_test = rate_pair_df.sample(100).reset_index(drop=True)

F_text, F_time, F_cat, F_newD_reviewerID, F_newD_itemID, _, _, _ = preprocess(temp_train, temp_test, vocabulary_size)

y = temp_train.overall.to_numpy()
X = np.concatenate([F_text, F_time, F_cat, F_newD_reviewerID, F_newD_itemID], axis=-1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [113]:
########### naive_bayes
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
nb = MultinomialNB()
nb.fit(X_train, y_train)
nb_preds = nb.predict(X_test)
print('Naive_bayesian RMS=%f'%(metrics.mean_squared_error(y_test, nb_preds)))

Naive_bayesian RMS=1.356667


In [114]:
########### SVM
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
# Create an instance of Logistic Regression Classifier and fit the data.
logreg.fit(X_train, y_train)
rl_preds=logreg.predict(X_test)
print('Logistic Regression RMS=%f'%(metrics.mean_squared_error(y_test,rl_preds)))

Logistic Regression RMS=1.116667


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [125]:
# x = preprocess(temp_train, temp_test, vocabulary_size)


8

In [ ]:
MaxWordLength=50
Batch_Size = 10000
Dp_rate=0.5
Epochs=10
vocabulary_size = 5000

temp_train = df.sample(1000).reset_index(drop=True)
temp_test = rate_pair_df.sample(100).reset_index(drop=True)

F_text, F_time, F_cat, F_newD_reviewerID, F_newD_itemID, F_newD_itemID_te, vocab_size_RID, vocab_size_IID = preprocess(temp_train, temp_test, vocabulary_size)

''' Model Selection'''
x_text = Input(shape=(F_text.shape[1]),name='x_text')
x_time = Input(shape=(F_time.shape[1]),name='x_time')
x_cat = Input(shape=(F_cat.shape[1]),name='x_cat')
x_RID = Input(shape=(F_newD_reviewerID.shape[1]),name='x_RID')
x_IID = Input(shape=(F_newD_itemID_te.shape[1]),name='x_IID')

H_t=tf.keras.layers.Embedding(vocabulary_size, 100)(x_text)
H_t=LSTM(100, dropout=Dp_rate, recurrent_dropout=Dp_rate)(H_t)

H_RID=tf.keras.layers.Embedding(vocab_size_RID, 20)(x_RID)
H_RID=tf.reshape(H_RID, (-1,20))

H_IID=tf.keras.layers.Embedding(vocab_size_IID, 20)(x_IID)
H_IID=tf.reshape(H_IID, (-1,20))

H_c=Dense(5,activation='relu')(x_cat)
H_c=Dropout(Dp_rate)(H_c)

H=tf.concat([H_t,H_RID,H_IID,x_cat,x_time],axis=-1)

# Final predictions and model.
prediction = Dense(1, activation='sigmoid')(H)
model = Model(inputs=[x_text,x_time,x_cat,x_RID,x_IID], outputs= prediction)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999)
model.compile(loss=tf.keras.losses.MSE,
              optimizer=optimizer,metrics=['mse'])
model.summary()

''' Train mode'''
history=model.fit([F_text,F_time,F_cat,F_newD_reviewerID,F_newD_itemID], y/5,
          batch_size=Batch_Size,
          epochs=Epochs,
          verbose=1,
          validation_split=.3, shuffle=True)
plt.figure(figsize=(12,5))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
x_text (InputLayer)             [(None, 5000)]       0                                            
__________________________________________________________________________________________________
x_RID (InputLayer)              [(None, 5)]          0                                            
__________________________________________________________________________________________________
x_IID (InputLayer)              [(None, 4)]          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 5000, 100)    500000      x_text[0][0]                     
______________________________________________________________________________________________

In [ ]:
''' Model Selection'''
x_text = Input(shape=(F_text.shape[1]),name='x_text')
x_time = Input(shape=(F_time.shape[1]),name='x_time')
x_cat = Input(shape=(F_cat.shape[1]),name='x_cat')
x_RID = Input(shape=(F_newD_reviewerID.shape[1]),name='x_RID')
x_IID = Input(shape=(F_newD_itemID_te.shape[1]),name='x_IID')

H_t=tf.keras.layers.Embedding(vocabulary_size, 100)(x_text)
H_t=LSTM(100, dropout=Dp_rate, recurrent_dropout=Dp_rate)(H_t)


H_RID=tf.keras.layers.Embedding(vocab_size_RID, 20)(x_RID)
H_RID=tf.reshape(H_RID, (-1,20))


H_IID=tf.keras.layers.Embedding(vocab_size_IID, 20)(x_IID)
H_IID=tf.reshape(H_IID, (-1,20))

H_c=Dense(5,activation='relu')(x_cat)
H_c=Dropout(Dp_rate)(H_c)

H=tf.concat([H_t,H_RID,H_IID,x_cat,x_time],axis=-1)

# Final predictions and model.
prediction = Dense(1, activation='sigmoid')(H)
model = Model(inputs=[x_text,x_time,x_cat,x_RID,x_IID], outputs= prediction)
optimizer = Adam(lr=0.01, beta_1=0.9, beta_2=0.999)
model.compile(loss=tf.keras.losses.MSE,
              optimizer=optimizer,metrics=['mse'])
model.summary()

''' Train mode'''
history=model.fit([F_text,F_time,F_cat,F_newD_reviewerID,F_newD_itemID], y/5,
          batch_size=Batch_Size,
          epochs=Epochs,
          verbose=1,
          validation_split=.3, shuffle=True)
plt.figure(figsize=(12,5))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()

In [ ]:
Text=[]
for i in range(Data.shape[0]):
    # print(i)
    Text.append(clean_text(str(Data['summary'][i]) + str(Data['reviewText'][i]) ))
for i in range(RatePairData.shape[0]):

    Text.append(clean_text(str(RatePairData['summary'][i]) + str(RatePairData['reviewText'][i]) ))

tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(Text)
F_textD = tokenizer.texts_to_matrix(Text, mode='freq')
# F_textD = pad_sequences(sequences, maxlen=MaxWordLength)
F_text=F_textD[:Data.shape[0],:]
F_text_te=F_textD[Data.shape[0]:,:]
'''P2--> Normalize unixReviewTime'''
F_time=( Data.unixReviewTime.values  - Data.unixReviewTime.min())/(Data.unixReviewTime.max() - Data.unixReviewTime.min())
F_time=F_time.reshape([-1,1])
'''P3--> one-hot encoding of categories'''
F_cat=pd.get_dummies(Data.category).values
'''P4 --> for later'''
''' Part 5 --> add popularity of an product as feature. I used histigram method'''
newD=pd.DataFrame(np.concatenate([Data['itemID'].values,RatePairData['itemID'].values],axis=0),columns=['itemID'])
newD['reviewerID']=np.concatenate([Data['reviewerID'].values,RatePairData['reviewerID'].values],axis=0)
newD['itemID']=newD['itemID'].groupby(newD['itemID']).transform('count')
newD['itemID']=(newD['itemID'].values-newD['itemID'].min())/(newD['itemID'].max()-newD['itemID'].min())

newD['reviewerID']=newD['reviewerID'].groupby(newD['reviewerID']).transform('count')
newD['reviewerID']=(newD['reviewerID'].values-newD['reviewerID'].min())/(newD['reviewerID'].max()-newD['itemID'].min())
newD=newD.to_numpy()
F_newD=newD[:Data.shape[0]]
F_newD_te=newD[Data.shape[0]:]

'''test-train split '''
y = Data['overall']
y=Data.overall.to_numpy()

''' Model Selection'''
x_text = Input(shape=(F_text.shape[1]),name='x_text')
x_time = Input(shape=(F_time.shape[1]),name='x_time')
x_cat = Input(shape=(F_cat.shape[1]),name='x_cat')
x_newD = Input(shape=(F_newD.shape[1]),name='x_newD')


H=Dense(MaxWordLength,activation='relu')(x_text)
H=Dropout(Dp_rate)(H)
H=Dense(MaxWordLength,activation='relu')(H)
H=Dropout(Dp_rate)(H)
H=tf.concat([H,x_time,x_cat,x_newD],axis=-1)
H=Dense(MaxWordLength, activation='relu')(H)
H=Dense(MaxWordLength, activation='relu')(H)
# Final predictions and model.
prediction = Dense(1, activation='sigmoid')(H)
# kernel_initializer='ones',
#     kernel_regularizer=tf.keras.regularizers.L1(0.01),
#     activity_regularizer=tf.keras.regularizers.L2(0.01)
model = Model(inputs=[x_text,x_time,x_cat,x_newD], outputs= prediction)
model.compile(loss=tf.keras.losses.MSE,
              optimizer='adam')
model.summary()

''' Train modek'''
history=model.fit([F_text,F_time,F_cat,F_newD], y/5,
          batch_size=Batch_Size,
          epochs=Epochs,
          verbose=1,
          validation_split=0.0,shuffle=True)
plt.figure(figsize=(12,5))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.show()
##################### prediction
'''P2--> Normalize unixReviewTime'''
F_time_te=( RatePairData.unixReviewTime.values  - Data.unixReviewTime.min())/(Data.unixReviewTime.max() - Data.unixReviewTime.min())
F_time_te=F_time_te.reshape([-1,1])
'''P3--> one-hot encoding of categories'''
F_cat_te=pd.get_dummies(RatePairData.category).values

y_pred = model([F_text_te,F_time_te,F_cat_te,F_newD_te])
RatePairID.prediction=y_pred.numpy()*5


RatePairID.to_csv('rating_predictions.csv',index=False)